In [1]:
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import ComplementNB
import lightgbm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import pickle

from sklearn.metrics import confusion_matrix,f1_score,roc_auc_score,roc_curve
import matplotlib.pyplot as plt

from tqdm import tqdm 

%matplotlib inline

/Users/shunling/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


# Feature Engineering

In [2]:
df = pd.read_csv('Processed_data/feature_engineered.csv',index_col=0)

In [3]:
df.head()

,claim_id,enrollee_id,enrollee_cum_claim_count,enrollee_cum_claim_amount,provider_id,hmo_id,care_id,diagnosis_id,qty,amount,unit_price,tariffs,create_m,vetted_m,label
0,10.0,89.0,1,1816.08,1.0,1.0,586.0,9.0,6.0,1816.08,302.68,302.68,3,5,0
1,11.0,89.0,2,3632.16,1.0,1.0,586.0,15925.0,6.0,1816.08,302.68,302.68,3,5,0
2,13.0,74.0,1,115.00,1.0,1.0,434.0,4342.0,5.0,115.00,23.00,23.00,3,5,0
3,13.0,74.0,2,1380.00,1.0,1.0,1102.0,4342.0,10.0,1265.00,126.50,126.50,3,5,0
4,13.0,74.0,3,1518.00,1.0,1.0,299.0,4342.0,15.0,138.00,9.20,9.20,3,5,0


In [3]:
df['tariffs'] = df['tariffs'].fillna(0)

In [4]:
df.dropna(inplace=True)

In [5]:
# training and testing data

X = df.drop(columns=['claim_id','enrollee_id','vetted_m','label'])


In [7]:
X.head()

,enrollee_cum_claim_count,enrollee_cum_claim_amount,provider_id,hmo_id,care_id,diagnosis_id,qty,amount,unit_price,tariffs,create_m
0,1,1816.08,1.0,1.0,586.0,9.0,6.0,1816.08,302.68,302.68,3
1,2,3632.16,1.0,1.0,586.0,15925.0,6.0,1816.08,302.68,302.68,3
2,1,115.00,1.0,1.0,434.0,4342.0,5.0,115.00,23.00,23.00,3
3,2,1380.00,1.0,1.0,1102.0,4342.0,10.0,1265.00,126.50,126.50,3
4,3,1518.00,1.0,1.0,299.0,4342.0,15.0,138.00,9.20,9.20,3


In [8]:
columns = ['enrollee_cum_claim_count', 'enrollee_cum_claim_amount', 'provider_id',
       'care_id', 'diagnosis_id', 'qty', 'amount', 'unit_price',
       'tariffs', 'create_m']

In [9]:
# create feature correlation features
for i in tqdm(range(len(columns))):
    col1 = columns[i]
    for col2 in columns[i+1:]:
        new = []
        pairs = {}
        n = 1
        for x,y in zip(df[col1],df[col2]):
            if (x,y) not in pairs:
                pairs[(x,y)] = n
                n += 1
            new.append(pairs[(x,y)])
                
            
        X[col1+"&"+col2] = new

100%|██████████| 10/10 [00:47<00:00,  4.79s/it]


In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744861 entries, 0 to 744869
Data columns (total 56 columns):
enrollee_cum_claim_count                              744861 non-null int64
enrollee_cum_claim_amount                             744861 non-null float64
provider_id                                           744861 non-null float64
hmo_id                                                744861 non-null float64
care_id                                               744861 non-null float64
diagnosis_id                                          744861 non-null float64
qty                                                   744861 non-null float64
amount                                                744861 non-null float64
unit_price                                            744861 non-null float64
tariffs                                               744861 non-null float64
create_m                                              744861 non-null int64
enrollee_cum_claim_count&enrollee_c

In [11]:
X.head()

,enrollee_cum_claim_count,enrollee_cum_claim_amount,provider_id,hmo_id,care_id,diagnosis_id,qty,amount,unit_price,tariffs,...,qty&amount,qty&unit_price,qty&tariffs,qty&create_m,amount&unit_price,amount&tariffs,amount&create_m,unit_price&tariffs,unit_price&create_m,tariffs&create_m
0,1,1816.08,1.0,1.0,586.0,9.0,6.0,1816.08,302.68,302.68,...,1,1,1,1,1,1,1,1,1,1
1,2,3632.16,1.0,1.0,586.0,15925.0,6.0,1816.08,302.68,302.68,...,1,1,1,1,1,1,1,1,1,1
2,1,115.00,1.0,1.0,434.0,4342.0,5.0,115.00,23.00,23.00,...,2,2,2,2,2,2,2,2,2,2
3,2,1380.00,1.0,1.0,1102.0,4342.0,10.0,1265.00,126.50,126.50,...,3,3,3,3,3,3,3,3,3,3
4,3,1518.00,1.0,1.0,299.0,4342.0,15.0,138.00,9.20,9.20,...,4,4,4,4,4,4,4,4,4,4


In [13]:
y = df['label']

In [14]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.2, random_state=0)

In [16]:
def init_scores():
    
    scores = {}
    scores['f1_socre'] = []
    scores['precision'] = []
    scores['recall'] = []
    scores['FPR'] = []
    scores['specificity'] = []
    scores['roc_auc'] = []
    
    return scores


def evaluation(ytest,Xtest,cls):

    scores = init_scores()
    ypred = cls.predict(Xtest)
    C = confusion_matrix(ytest,ypred)
    TN = C[0][0]
    FN = C[1][0]
    TP = C[1][1]
    FP = C[0][1]

    print('f1_socre: {:0.3f}'.format(f1_score(ytest,ypred)))
    scores['f1_socre'].append(f1_score(ytest,ypred))
    print('precision: {:0.3f}'.format(TP/(TP+FP)))
    scores['precision'].append(TP/(TP+FP))
    print('recall/sensitivity(true positive rate): {:0.3f}'.format(TP/(TP+FN)))
    scores['recall'].append(TP/(TP+FN))
    print('false positive rate (FPR): {:0.3f}'.format(1-(TN/(TN+FP)))) # 1 - specificity
    scores['FPR'].append(1-(TN/(TN+FP)))
    print('spcificity(true negative rate): {:0.3f}'.format(TN/(TN+FP)))
    scores['specificity'].append(TN/(TN+FP))
    print('ROC_AUC_score: {:0.3f}'.format(roc_auc_score(ytest,ypred)))
    scores['roc_auc'].append(roc_auc_score(ytest,ypred))
    
    return scores
    
def cv(Xtrain,ytrain,model):

    models = []
    kf = KFold(n_splits=4)
    print(model)
    n = 0
    for train_index, test_index in kf.split(Xtrain):
        print('cross_validate_run: {}'.format(n))
        Xtr, Xte = Xtrain[train_index], Xtrain[test_index]
        ytr, yte = ytrain[train_index], ytrain[test_index]
        cls = model.fit(Xtr, ytr) 
        models.append(cls)
        scores = evaluation(yte,Xte,cls)
        n += 1
    
    print('\n mean scores +/- sd: \n')
    for k in scores:
        print('{} : {:0.3f} +/- {:0.3f}'.format(k, np.array(scores[k]).mean(),  np.array(scores[k]).std()))
        
    
    return models, scores

# Probability Solution

In [17]:
# tranform value of each feature according to their probability in two types of groups
def prob(col,df,labels): # A is the column value array
    counts = {}
    value = {} # normalized value in total
    total = {} # total claim items each feature pair
    dif = {} # diference between false claim (1) and legal claim (0)
    
    for i,label in zip(df[col],labels):
        if i in counts:
            if label:
                counts[i][1] += 1
            else:
                counts[i][0] += 1
        else:
            if label:
                counts[i] = {1:1,0:0}
            else:
                counts[i] = {1:0,0:1}
        total[i] = counts[i][1] + counts[i][0]
        dif[i] = counts[i][1] - counts[i][0]
        value[i] = dif[i]/total[i]
        
    return value

In [18]:
Xtrain.to_csv('Processed_data/Xtrain56.csv')

In [19]:
Xtest.to_csv('Processed_data/Xtest56.csv')

In [20]:
Rules = {}
for col in tqdm(Xtrain.columns):
    Rules[col] = prob(col,Xtrain,ytrain)

100%|██████████| 56/56 [00:54<00:00,  1.04it/s]


In [21]:
def convert_prob(df,Rules):
    
    col_prob = {}
    for col in tqdm(df.columns):
        col_prob[col] = []
        for x in df[col]:
            if x in Rules[col]:
                col_prob[col].append(Rules[col][x])
            else:
                col_prob[col].append(0)
    return col_prob

In [22]:
X_train = convert_prob(Xtrain,Rules)


100%|██████████| 56/56 [00:15<00:00,  3.71it/s]


In [26]:
X_train_df = pd.DataFrame(X_train)

In [23]:
X_test = convert_prob(Xtest,Rules)

100%|██████████| 56/56 [00:04<00:00, 12.89it/s]


In [28]:
X_test_df = pd.DataFrame(X_test)

In [60]:
probTrain = X_train_df.mean(axis=1)
probTrain = probTrain.apply(lambda x: x>=0)

In [63]:
probTest = X_test_df.mean(axis=1)
probTest = probTest.apply(lambda x: x>=0)

In [64]:
f1_score(ytrain,probTrain.values),f1_score(ytest,probTest.values)

(0.13434811383189937, 0.022422921208346314)

In [62]:
confusion_matrix(ytrain,probTrain.values)

array([[570517,      0],
       [ 23544,   1827]])

In [65]:
confusion_matrix(ytest,probTest.values)

array([[142623,      2],
       [  6276,     72]])